In [4]:
import pygame
import heapq
import random
import time
from pygame.locals import *

# -------------------- Classe Noeud pour la résolution automatique -------------------- #
class Noeud:
    def __init__(self, state, pred=None, g=0, h=0):
        self.state = state  # État actuel du puzzle
        self.pred = pred  # Noeud prédécesseur
        self.succ = []  # Successeurs
        self.g = g  # Coût accumulé
        self.h = h  # Estimation heuristique

    def getSucc(self):
        succ = []
        zero_index = self.state.index(0)
        row, col = divmod(zero_index, self.size)

        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Déplacements possibles : droite, gauche, bas, haut

        for dr, dc in moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < self.size and 0 <= new_col < self.size:  # Vérification des limites
                new_index = new_row * self.size + new_col
                new_state = self.state[:]
                new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
                succ.append(Noeud(new_state, pred=self))
        return succ

    def isSuccess(self):
        return self.state == list(range(1, self.size * self.size)) + [0]

    def __lt__(self, other):
        return (self.g + self.h) < (other.g + other.h)

    def __str__(self):
        return str(self.state)


def heuristic(node):
    goal = list(range(1, node.size * node.size)) + [0]
    distance = 0
    for i, value in enumerate(node.state):
        if value != 0:
            goal_index = goal.index(value)
            distance += abs(i // node.size - goal_index // node.size) + abs(i % node.size - goal_index % node.size)
    return distance


def a_star(depart):
    open_list = []
    closed_list = set()
    heapq.heappush(open_list, depart)

    while open_list:
        current = heapq.heappop(open_list)
        if current.isSuccess():
            return current

        closed_list.add(tuple(current.state))

        for succ in current.getSucc():
            if tuple(succ.state) not in closed_list:
                succ.g = current.g + 1
                succ.h = heuristic(succ)
                heapq.heappush(open_list, succ)

    return None


# -------------------- Interface graphique en Pygame -------------------- #
class Tile:
    def __init__(self, screen, value, x, y, size):
        self.screen = screen
        self.value = value
        self.x = x
        self.y = y
        self.size = size
        self.font = pygame.font.Font(None, 60)  # Taille de la police ajustée pour 4x4

    def draw(self):
        color = (200, 200, 200) if self.value != 0 else (50, 50, 50)
        pygame.draw.rect(self.screen, color, (self.x, self.y, self.size, self.size))
        if self.value != 0:
            text = self.font.render(str(self.value), True, (0, 0, 0))
            text_rect = text.get_rect(center=(self.x + self.size // 2, self.y + self.size // 2))
            self.screen.blit(text, text_rect)


class Game:
    def __init__(self, size=3):
        pygame.init()
        self.size = size
        self.screen_size = (self.size * 90 + 10, self.size * 90 + 110)  # Adjust screen size based on puzzle size
        self.screen = pygame.display.set_mode(self.screen_size)
        pygame.display.set_caption(f"Jeu de Taquin {self.size}x{self.size}")
        self.clock = pygame.time.Clock()
        self.tiles = []
        self.state = list(range(1, self.size * self.size)) + [0]
        self.goal = list(range(1, self.size * self.size)) + [0]
        self.shuffle_state()
        self.selected_tile = None

    def shuffle_state(self):
        random.shuffle(self.state)
        while not self.is_solvable():
            random.shuffle(self.state)

    def is_solvable(self):
        inversions = 0
        for i in range(len(self.state)):
            for j in range(i + 1, len(self.state)):
                if self.state[i] != 0 and self.state[j] != 0 and self.state[i] > self.state[j]:
                    inversions += 1
        return inversions % 2 == 0

    def draw_tiles(self):
        self.tiles = []
        for i, value in enumerate(self.state):
            x = (i % self.size) * 90 + 10  # Position ajustée pour la grille
            y = (i // self.size) * 90 + 10  # Position ajustée pour la grille
            tile = Tile(self.screen, value, x, y, 80)
            tile.draw()
            self.tiles.append(tile)

    def get_empty_index(self):
        return self.state.index(0)

    def move_tile(self, index):
        empty_index = self.get_empty_index()
        empty_row, empty_col = divmod(empty_index, self.size)
        row, col = divmod(index, self.size)

        if abs(row - empty_row) + abs(col - empty_col) == 1:
            self.state[empty_index], self.state[index] = self.state[index], self.state[empty_index]

    def solve_puzzle(self):
        start_node = Noeud(self.state, size=self.size)
        solution_node = a_star(start_node)

        if solution_node:
            path = []
            while solution_node:
                path.append(solution_node)
                solution_node = solution_node.pred
            path.reverse()
            for node in path:
                self.state = node.state
                self.draw_tiles()
                pygame.display.flip()
                time.sleep(0.3)

    def run(self):
        running = True
        while running:
            self.screen.fill((30, 30, 30))
            self.draw_tiles()

            for event in pygame.event.get():
                if event.type == QUIT:
                    running = False

                if event.type == MOUSEBUTTONDOWN:
                    x, y = event.pos
                    for i, tile in enumerate(self.tiles):
                        if tile.x < x < tile.x + tile.size and tile.y < y < tile.y + tile.size:
                            self.move_tile(i)

                if event.type == KEYDOWN:
                    if event.key == K_s:
                        self.solve_puzzle()

            if self.state == self.goal:
                font = pygame.font.Font(None, 60)
                text = font.render("Résolu !", True, (0, 255, 0))
                self.screen.blit(text, (120, self.screen_size[1] - 60))

            pygame.display.flip()
            self.clock.tick(30)


class Menu:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((400, 300))
        pygame.display.set_caption("Menu de sélection")
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 60)
        self.running = True

    def draw(self):
        self.screen.fill((30, 30, 30))
        text_3x3 = self.font.render("Puzzle 3x3", True, (255, 255, 255))
        text_4x4 = self.font.render("Puzzle 4x4", True, (255, 255, 255))
        self.screen.blit(text_3x3, (100, 100))
        self.screen.blit(text_4x4, (100, 200))

    def run(self):
        while self.running:
            for event in pygame.event.get():
                if event.type == QUIT:
                    self.running = False
                if event.type == MOUSEBUTTONDOWN:
                    x, y = event.pos
                    if 100 < x < 300 and 100 < y < 150:
                        Game(size=3).run()
                        self.running = False
                    elif 100 < x < 300 and 200 < y < 250:
                        Game(size=4).run()
                        self.running = False

            self.draw()
            pygame.display.flip()
            self.clock.tick(30)


if __name__ == "__main__":
    Menu().run()